In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

In [2]:
import numpy as np

# Get Plane

In [3]:
from Planes.hermes import hermes

airplane = hermes("hermes")
# airplane.plot()

# Import Environment and Set State

In [4]:
from ICARUS.environment import EARTH_ISA
from ICARUS.flight_dynamics import State

u_freestream = 20
unstick = State(
    name="unstick", airplane=airplane, environment=EARTH_ISA, u_freestream=u_freestream
)

In [46]:
import jax
import jax.numpy as jnp
from ICARUS.aero import LSPT_Plane, AerodynamicState
from ICARUS.aero.vlm import get_RHS, get_LHS

plane_LSPT = LSPT_Plane(plane=airplane)
# plane_LSPT.plot_panels(plot_wake= True)

aerodynamic_state = AerodynamicState(
    airspeed=20,
    altitude=0.,
    density= 1.225,
    mach=0.0,  # Assuming incompressible flow
    # Positional State
    alpha=0.0,
    beta=0.0,
    rate_P=0.0,
    rate_Q=0.0,
    rate_R=0.0,
)

Q = aerodynamic_state.velocity_vector_jax
RHS = get_RHS(plane_LSPT, Q)

A, A_star = get_LHS(plane_LSPT)
A_LU, A_piv = jax.scipy.linalg.lu_factor(A)

gammas = jax.scipy.linalg.lu_solve((A_LU, A_piv), RHS)
w_induced = jnp.matmul(A_star, gammas)


In [57]:
import matplotlib.pyplot as plt
gammas[abs(gammas) > 1e7]

Array([ 8.0970793e+14, -8.0970793e+14], dtype=float64)

In [58]:
from ICARUS.aero.vlm import run_vlm_analysis

run_vlm_analysis(
    plane=plane_LSPT,
    state= unstick,
    angles = np.linspace(-5, 5, 11),
)

Angle of Attack: -5.0 degrees
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
	Total Lift (Potential): 0.00 N
	Total Drag (Potential): 0.00 N
	Total Moment (Potential): 0.00 Nm
Angle of Attack: -4.0 degrees
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
	Total Lift (Potential): 0.00 N
	Total Drag (Potential): 0.00 N
	Total Moment (Potential): 0.00 Nm
Angle of Attack: -3.0 degrees
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
	Total Lift (Potential): 0.00 N
	Total Drag (Potential): 0.00 N
	Total Moment (Potential): 0.00 Nm
Angle of Attack: -2.0 degrees
0.0
0.0
0.0
0.0
0.0

,AoA,Lift_Potential,Drag_Potential,Lift_Viscous,Drag_Viscous,CL,CD,CL_2D,CD_2D
0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
loads.strips

[StripLoads(chord=0.157, width=0.059, num_panels=14, mean_gamma=0.000, mean_w_induced=0.000),
 StripLoads(chord=0.152, width=0.059, num_panels=14, mean_gamma=0.000, mean_w_induced=0.000),
 StripLoads(chord=0.148, width=0.059, num_panels=14, mean_gamma=0.000, mean_w_induced=0.000),
 StripLoads(chord=0.143, width=0.059, num_panels=14, mean_gamma=0.000, mean_w_induced=0.000),
 StripLoads(chord=0.138, width=0.059, num_panels=14, mean_gamma=0.000, mean_w_induced=0.000),
 StripLoads(chord=0.134, width=0.059, num_panels=14, mean_gamma=0.000, mean_w_induced=0.000),
 StripLoads(chord=0.129, width=0.059, num_panels=14, mean_gamma=0.000, mean_w_induced=0.000),
 StripLoads(chord=0.125, width=0.059, num_panels=14, mean_gamma=0.000, mean_w_induced=0.000),
 StripLoads(chord=0.120, width=0.059, num_panels=14, mean_gamma=0.000, mean_w_induced=0.000),
 StripLoads(chord=0.116, width=0.059, num_panels=14, mean_gamma=0.000, mean_w_induced=0.000),
 StripLoads(chord=0.111, width=0.059, num_panels=14, mean_ga

# Get Solver

In [8]:
from ICARUS.computation.solvers.Icarus_LSPT import LSPT

lspt = LSPT()

print(lspt)

3D VLM Solver LSPT:
Available Analyses Are: 
------------------- 
0) Aiplane Polar Analysis 



## AoA Run

In [9]:
analysis: str = lspt.get_analyses_names()[0]
print(f"Selecting Analysis: {analysis}")
lspt.select_analysis(analysis)

Selecting Analysis: Aiplane Polar Analysis


In [10]:
options = lspt.get_analysis_options(verbose=True)
solver_parameters = lspt.get_solver_parameters(verbose=True)

Aiplane Polar Analysis
Available Options of LSPT for Aiplane Polar Analysis: 

| VarName   | Value   | Description                                         |
|-----------|---------|-----------------------------------------------------|
| plane     | None    | Vehicle Airplane Object                             |
| state     | None    | State Object                                        |
| solver2D  | None    | Name of 2D Solver from which to use computed polars |
| angles    | None    | List of angles to run polars                        |

If there are Multiple Values, or complex datatypes, or N/A you should inspect them sepretly by calling the option name

Available Solver Parameters of LSPT for LSPT: 

| VarName        | Value           | Description                                                                        |
|----------------|-----------------|------------------------------------------------------------------------------------|
| Ground_Effect  | None            | Dis

In [ ]:
plane_LSPT.plot_surface_gamma_distribution()

In [ ]:
# Plot the panels and lift
plane_LSPT.plot_L_pan()
plane_LSPT.plot_D_pan()

In [ ]:
plane_LSPT.L_pan

In [ ]:
AoAmin = -6
AoAmax = 8
NoAoA = (AoAmax - AoAmin) + 1
angles = np.linspace(AoAmin, AoAmax, NoAoA, dtype=float)
# airplane.define_dynamic_pressure(u_freestream, EARTH.air_density)

options.plane = airplane
options.state = unstick
options.solver2D = "Xfoil"
# options.maxiter       = 100
# options.timestep      = 0.001
options.angles = angles

solver_parameters.Use_Grid = 0

lspt.define_analysis(options, solver_parameters)
lspt.print_analysis_options()
lspt.execute(parallel=False)
polars = lspt.get_results()
airplane.save()

# Dynamics

### Define and Trim Plane

In [ ]:
from ICARUS.flight_dynamics import State

if isinstance(polars, int):
    raise Exception("No Polars")
unstick = State("Unstick", airplane, environment=EARTH_ISA, u_freestream=u_freestream)
unstick.add_polar(polars)

### Pertrubations

In [ ]:
unstick.add_all_pertrubations("Central")
unstick.get_pertrub()

In [ ]:
analysis = lspt.get_analyses_names(verbose=True)[1]  # ANGLES PARALLEL
print(f"Selecting Analysis: {analysis}")
lspt.select_analysis(analysis)
options = lspt.get_analysis_options(verbose=True)

In [ ]:
options.plane = airplane
options.state = unstick
options.solver2D = "XFLR"
options.maxiter = 200
options.timestep = 5e-3

lspt.print_analysis_options()
lspt.define_analysis(options, solver_parameters)
lspt.execute(parallel=True)
lspt.get_results()